<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e33a1fa17b7619e8fc64dfcc4f7a42ae3130e855502670d4d6b907d7e23acbbe
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-14 10:41:36
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.45 C
-------------------
Today PnL: -26.59 K (-0.18%)
Current PnL: -20.44 L (-13.39%)
CY Booked + Current PnL: -6.92 L (-4.53%)
-------------------
Total profit:  1.88 L
Total loss:  -22.32 L
-------------------
Total Booked + Current PnL: 20.06 L (15.97%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.33%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 91.32 L (62.98%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,144.42,84.0,M-SC,1.92,160450.0,-14109.0,14136.0,0.28,-8.08,8.81,0.02,234.0,-1.00,1.11,39.56,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.28,50.0,M-SC,3.08,86683.0,-14094.0,14190.0,0.32,-13.99,16.37,0.09,253.0,-0.99,0.60,13.88,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.43,88665.0,-2013.0,16917.0,-0.05,-2.22,19.08,16.44,63.0,-0.12,0.61,17.51,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.55,56.0,X-MC,7.50,187739.0,13419.0,21778.0,-0.74,7.70,11.60,20.19,68.0,0.62,1.30,23.98,X40,ATH,INSURANCE
43,ITC,452.00,-41.15,42.0,X-LC,0.90,197678.0,-2460.0,23346.0,-0.36,-1.23,11.81,10.44,5.0,-0.11,1.36,4.27,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,38.92,73.0,H-SC,15.21,286416.0,5293.0,127570.0,5.12,1.88,44.54,47.26,171.0,0.04,1.98,53.45,XY24,NTT,INSURANCE
83,VOLTAS,1530.00,-4.12,48.0,X-MC,3.80,203715.0,11973.0,25790.0,1.59,6.24,12.66,19.69,78.0,0.46,1.41,13.47,XY25,NTT,AC
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,9.30,114540.0,-258.0,94301.0,1.44,-0.22,82.33,81.92,236.0,-0.00,0.79,35.60,XR,NTT,DURABLES
5,ANGELONE,3033.00,58.43,72.0,X-SC,7.07,222507.0,31501.0,26189.0,1.27,16.49,11.77,30.21,97.0,1.20,1.54,40.82,X40N,NTT,FINANCE
82,VBL,671.64,-18.42,47.0,X-LC,6.07,292172.0,-23670.0,136327.0,1.09,-7.49,46.66,35.67,4.0,-0.17,2.02,5.32,X40N,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,2275.00,-20.32,53.0,X-LC,4.21,320160.0,7272.0,164402.0,-2.51,2.32,51.35,54.87,2.0,0.04,2.21,10.80,X40,BTT,IT
77,TRIDENT,48.00,-10.90,45.0,M-SC,1.90,69432.0,-22822.0,49130.0,-2.29,-24.74,70.76,28.51,225.0,-0.46,0.48,18.81,XR,NTT,TEXTILES
68,SONACOMS,806.63,-35.30,54.0,M-SC,6.36,83729.0,-17430.0,57430.0,-2.19,-17.23,68.59,39.54,220.0,-0.30,0.58,18.07,AR,ATH,AUTO
64,SFL,1287.00,27.48,39.0,M-SC,3.03,168397.0,-93821.0,163648.0,-2.13,-35.78,97.18,26.63,239.0,-0.57,1.16,4.70,XY24,NTT,MISC
14,BSOFT,831.70,-10.32,60.0,H-SC,8.92,98302.0,-42351.0,113785.0,-2.01,-30.11,115.75,50.79,151.0,-0.37,0.68,14.46,XR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,9.30,114540.0,-258.0,94301.0,1.44,-0.22,82.33,81.92,236.0,-0.00,0.79,35.60,XR,NTT,DURABLES
25,FINCABLES,1641.55,-19.34,45.0,M-SC,10.35,146702.0,-942.0,160272.0,-0.61,-0.64,109.25,107.91,216.0,-0.01,1.01,2.34,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.75,208414.0,-1160.0,75821.0,-0.12,-0.55,36.38,35.63,119.0,-0.02,1.44,16.03,AR,ATH,PHARMA
85,WIPRO,420.00,-13.79,55.0,M-LC,5.45,152105.0,1160.0,108299.0,0.05,0.77,71.20,72.51,99.0,0.01,1.05,6.84,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-19.34,45.0,M-SC,10.35,146702.0,-942.0,160272.0,-0.61,-0.64,109.25,107.91,216.0,-0.01,1.01,2.34,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,144.42,84.0,M-SC,1.92,160450.0,-14109.0,14136.0,0.28,-8.08,8.81,0.02,234.0,-1.00,1.11,39.56,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.28,50.0,M-SC,3.08,86683.0,-14094.0,14190.0,0.32,-13.99,16.37,0.09,253.0,-0.99,0.60,13.88,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.17,47.0,H-SC,2.44,222237.0,-47430.0,83761.0,-0.97,-17.59,37.69,13.47,158.0,-0.57,1.53,13.00,XY24,NTT,PAINTS
67,SIS,528.00,2019.69,47.0,H-SC,2.43,85540.0,-25492.0,49100.0,-0.13,-22.96,57.40,21.26,166.0,-0.52,0.59,15.57,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,38.92,73.0,H-SC,15.21,286416.0,5293.0,127570.0,5.12,1.88,44.54,47.26,171.0,0.04,1.98,53.45,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,17.65,53.0,M-SC,36.48,159300.0,10080.0,134099.0,-0.75,6.76,84.18,96.62,245.0,0.08,1.10,72.40,XY25,NTT,FINANCE
85,WIPRO,420.00,-13.79,55.0,M-LC,5.45,152105.0,1160.0,108299.0,0.05,0.77,71.20,72.51,99.0,0.01,1.05,6.84,XR,NTT,IT
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.62,131053.0,7717.0,123911.0,-0.18,6.26,94.55,106.72,139.0,0.06,0.90,31.10,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.62,131053.0,7717.0,123911.0,-0.18,6.26,94.55,106.72,139.0,0.06,0.90,31.10,AR,ATH,MISC
85,WIPRO,420.00,-13.79,55.0,M-LC,5.45,152105.0,1160.0,108299.0,0.05,0.77,71.20,72.51,99.0,0.01,1.05,6.84,XR,NTT,IT
84,WHIRLPOOL,2270.00,-41.57,40.0,M-SC,9.30,114540.0,-258.0,94301.0,1.44,-0.22,82.33,81.92,236.0,-0.00,0.79,35.60,XR,NTT,DURABLES
25,FINCABLES,1641.55,-19.34,45.0,M-SC,10.35,146702.0,-942.0,160272.0,-0.61,-0.64,109.25,107.91,216.0,-0.01,1.01,2.34,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.53,37.0,H-MC,5.75,208414.0,-1160.0,75821.0,-0.12,-0.55,36.38,35.63,119.0,-0.02,1.44,16.03,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,-18.02,28.0,X-SC,5.91,253290.0,-606.0,82902.0,-0.24,-0.24,32.73,32.42,85.0,-0.01,1.75,14.82,X40,ATH,FMCG
55,QUESS,986.00,-53.17,28.0,X-SC,37.29,46597.0,-18409.0,169338.0,-0.96,-28.32,363.41,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
27,GLAXO,3466.20,-19.47,29.0,X-MC,5.82,199800.0,516.0,77502.0,0.12,0.26,38.79,39.15,60.0,0.01,1.38,29.34,X40,ATH,PHARMA
12,BATAINDIA,2096.00,13.94,30.0,X-SC,8.94,86851.0,-41819.0,87120.0,-1.05,-32.50,100.31,35.20,92.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,-19.06,32.0,X-MC,5.37,189776.0,-10571.0,41068.0,-0.07,-5.28,21.64,15.23,66.0,-0.26,1.31,25.27,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-15.75,40.0,X-LC,0.12,258561.0,-17849.0,439011.0,-0.80,-6.46,169.79,152.37,3.0,-0.04,1.78,1.00,XY24,NTT,AUTO
36,ICICIPRULI,790.00,-20.11,67.0,X-MC,0.77,184619.0,7881.0,46856.0,-0.45,4.46,25.38,30.97,75.0,0.17,1.27,17.58,X40,ATH,INSURANCE
43,ITC,452.00,-41.15,42.0,X-LC,0.90,197678.0,-2460.0,23346.0,-0.36,-1.23,11.81,10.44,5.0,-0.11,1.36,4.27,X40,NTT,FMCG
34,HONAUT,58357.33,-24.86,37.0,X-SC,0.95,106755.0,-21183.0,68313.0,0.20,-16.56,63.99,36.84,90.0,-0.31,0.74,9.29,X40N,ATH,ELECTRICAL
16,CAMS,4762.00,-1.02,58.0,X-SC,1.63,261360.0,9504.0,52925.0,0.41,3.77,20.25,24.79,86.0,0.18,1.80,27.79,X40N,NTT,MISC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,13.94,30.0,X-SC,8.94,86851.0,-41819.0,87120.0,-1.05,-32.50,100.31,35.20,92.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,28.0,X-SC,37.29,46597.0,-18409.0,169338.0,-0.96,-28.32,363.41,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
53,PAGEIND,51769.94,-28.95,36.0,X-MC,7.70,158060.0,-5680.0,49014.0,-0.18,-3.47,31.01,26.47,55.0,-0.12,1.09,0.14,X40,ATH,APPARELS
76,TMPV,1065.00,-15.75,40.0,X-LC,0.12,258561.0,-17849.0,439011.0,-0.80,-6.46,169.79,152.37,3.0,-0.04,1.78,1.00,XY24,NTT,AUTO
54,PGHH,17907.65,-31.84,32.0,X-MC,1.90,193905.0,-6915.0,74712.0,0.19,-3.44,38.53,33.76,57.0,-0.09,1.34,1.57,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.17,28.0,X-SC,37.29,46597.0,-18409.0,169338.0,-0.96,-28.32,363.41,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-41.92,36.0,X-SC,3.47,79139.0,-66021.0,141952.0,-2.00,-45.48,179.37,52.31,91.0,-0.47,0.55,5.93,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,13.94,30.0,X-SC,8.94,86851.0,-41819.0,87120.0,-1.05,-32.50,100.31,35.20,92.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.14,47.0,X-MC,3.43,88665.0,-2013.0,16917.0,-0.05,-2.22,19.08,16.44,63.0,-0.12,0.61,17.51,X40,ATH,PHARMA
34,HONAUT,58357.33,-24.86,37.0,X-SC,0.95,106755.0,-21183.0,68313.0,0.20,-16.56,63.99,36.84,90.0,-0.31,0.74,9.29,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-26.86,56.0,X-LC,8.65,289717.0,-56239.0,122927.0,-0.76,-16.26,42.43,19.28,1.0,-0.46,2.00,7.10,X40,ATH,IT
41,INFY,2275.00,-20.32,53.0,X-LC,4.21,320160.0,7272.0,164402.0,-2.51,2.32,51.35,54.87,2.0,0.04,2.21,10.80,X40,BTT,IT
76,TMPV,1065.00,-15.75,40.0,X-LC,0.12,258561.0,-17849.0,439011.0,-0.80,-6.46,169.79,152.37,3.0,-0.04,1.78,1.00,XY24,NTT,AUTO
82,VBL,671.64,-18.42,47.0,X-LC,6.07,292172.0,-23670.0,136327.0,1.09,-7.49,46.66,35.67,4.0,-0.17,2.02,5.32,X40N,ATH,FMCG
43,ITC,452.00,-41.15,42.0,X-LC,0.90,197678.0,-2460.0,23346.0,-0.36,-1.23,11.81,10.44,5.0,-0.11,1.36,4.27,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-708.10,78.0,L-MC,6.48,49259.0,-27948.0,53343.0,0.09,-36.20,108.29,32.89,259.0,-0.52,0.34,35.66,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,21.78,76435.0,-37114.0,77161.0,-1.53,-32.69,100.95,35.27,270.0,-0.48,0.53,87.08,XR,NTT,HOTELS
7,ASIANTILES,137.00,7450.00,60.0,L-SC,16.34,82850.0,-10960.0,87581.0,-0.70,-11.68,105.71,81.67,271.0,-0.13,0.57,60.44,XR,NTT,CERAMICS
51,MASFIN,398.61,-19.60,48.0,H-SC,7.46,91545.0,-6435.0,28040.0,-0.36,-6.57,30.63,22.05,164.0,-0.23,0.63,33.07,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.62,131053.0,7717.0,123911.0,-0.18,6.26,94.55,106.72,139.0,0.06,0.90,31.10,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.49,86.0,X-LC,8.83,255534.0,3766.0,49446.0,0.85,1.50,19.35,21.13,31.0,0.08,1.76,37.45,X40,ATH,PAINTS
39,INDIGOPNTS,1408.00,144.42,84.0,M-SC,1.92,160450.0,-14109.0,14136.0,0.28,-8.08,8.81,0.02,234.0,-1.00,1.11,39.56,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,70.25,58.0,H-SC,8.40,147514.0,-35261.0,146334.0,-1.41,-19.29,99.20,60.77,159.0,-0.24,1.02,32.85,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7450.00,60.0,L-SC,16.34,82850.0,-10960.0,87581.0,-0.70,-11.68,105.71,81.67,271.0,-0.13,0.57,60.44,XR,NTT,CERAMICS
40,INDUSINDBK,1800.00,-708.10,78.0,L-MC,6.48,49259.0,-27948.0,53343.0,0.09,-36.20,108.29,32.89,259.0,-0.52,0.34,35.66,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.63
1,25,44.51
2,50,75.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.60
MC    27.73
LC    24.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.49
X40      20.51
X40N     12.27
XY25     10.07
XR        9.56
AR        8.98
OX40N     7.54
X200      1.80
SR        1.01
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.22
X-MC    21.38
X-LC    19.83
M-SC    13.37
X-SC     8.57
H-MC     4.65
H-LC     2.86
L-SC     1.44
M-MC     1.36
M-LC     1.05
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.62,-4.59,38.41
IT,12.48,-16.24,78.38
FINANCE,11.92,-9.47,59.09
MISC,7.55,-18.60,78.42
PAINTS,6.01,-5.93,21.72
ELECTRICAL,5.55,-12.50,52.39
INSURANCE,4.55,4.04,29.78
AUTO,3.46,-14.48,124.24
PHARMA,3.43,-0.53,34.26


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3318812.0,20
XR,1272162.0,13
AR,1270809.0,10
X40,996862.0,14
OX40N,700427.0,10
X40N,693902.0,9
XY25,476866.0,7
SR,274101.0,2
MH,65043.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3495365.0,25
M-SC,1413304.0,16
X-LC,1194020.0,11
X-MC,1146583.0,15
X-SC,834873.0,8
H-MC,405132.0,3
L-SC,253064.0,3
H-LC,142562.0,2
M-LC,108299.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1171554.0      6
           AR         843624.0      5
           XR         802347.0      7
M-SC       XY24       735975.0      6
X-LC       XY24       501751.0      2
X-MC       X40        414284.0      7
X-LC       X40        412556.0      5
X-SC       XY24       375472.0      2
X-MC       XY24       349716.0      3
H-SC       OX40N      338696.0      4
X-SC       X40N       289379.0      4
H-SC       SR         274101.0      2
M-SC       OX40N      273409.0      5
H-MC       AR         220788.0      2
X-LC       X40N       203595.0      2
X-MC       X40N       200928.0      3
H-MC       XY24       184344.0      1
X-MC       XY25       181655.0      2
X-SC       X40        170022.0      2
L-SC       XR         164742.0      2
M-SC       XR         143431.0      2
           XY25       134099.0      1
           AR         126390.0      2
M-LC       XR         108299.0      1
L-SC       OX40N       88322.0      1
H-LC       AR          80007.0      1
X-LC       XY25        76118.0      2
H-SC       MH          65043.0      1
H-LC       X200        62555.0      1
L-MC       XR          53343.0      1
M-MC       XY25        47651.0      1
L-LC       XY25        37343.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
